<a href="https://colab.research.google.com/github/appauldev/dezoomcamp2024/blob/main/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

sum = 0
generator5 = square_root_generator(5)
for sqrt_value in generator5:
    sum += sqrt_value

print(f"sum: {sum}")

generator13 = square_root_generator(13)
for val in generator13:
  print(val)

sum: 8.382332347441762
1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [7]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

# SEE SOLUTION BELOW #3


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [5]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [8]:
# to do: homework :)
# QUESTION 2 SOLUTIONS
import dlt

# create the duckdb schema and table
pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework')
# run the pipeline. Use replace for loading people_1 so we could reuse this code block without duplication issues
info = pipeline.run(people_1(), table_name="people", write_disposition="replace")
# print(info)
# append people_2
pipeline.run(people_2(), table_name="people", write_disposition="append")


LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7e6196909d80>, metrics={'1708357558.4640212': [{'started_at': DateTime(2024, 2, 19, 15, 45, 58, 845249, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 19, 15, 45, 59, 203238, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////content/dlt_colab_kernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='homework', loads_ids=['1708357558.4640212'], load_packages=[LoadPackageInfo(load_id='1708357558.4640212', package_path='/var/dlt/pipelines/dlt_colab_kernel_launcher/load/loaded/1708357558.4640212', state='loaded', schema=Schema dlt_colab_kernel_launcher at 138957513554480, schema_update={'people': {'name': 'people', 'columns': {'occupation': {'name': 'occupation', 'data_type': 'text', 'nullable': True}}, 'write_disposition

In [9]:
# QUESTION 2 SOLUTIONS continued

# view the table created in the previous block
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
# the dataset_name of the pipeline is the schema for the table created
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
# select and display all the loaded data
people_all = conn.sql("SELECT * FROM people").df()
display(people_all)
# get the sum of the ages
sum = conn.sql("SELECT sum(age) AS sum FROM people")
display(sum)


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708357557.5641918,OnOA97rOtMHjcw,None
1,2,Person_2,27,City_A,1708357557.5641918,um9f4auo9WQ2xg,None
2,3,Person_3,28,City_A,1708357557.5641918,ypH6K8ZwI6YwfA,None
3,4,Person_4,29,City_A,1708357557.5641918,09zbsAN6s7g3Kw,None
4,5,Person_5,30,City_A,1708357557.5641918,GKiKUZpPo80nXA,None
5,3,Person_3,33,City_B,1708357558.4640212,kKE1XOEK2qEQ4w,Job_3
6,4,Person_4,34,City_B,1708357558.4640212,AZTEpCDH1tjzww,Job_4
7,5,Person_5,35,City_B,1708357558.4640212,ErsTKCCoRsk2Jw,Job_5
8,6,Person_6,36,City_B,1708357558.4640212,coyPUq6OqfonbA,Job_6
9,7,Person_7,37,City_B,1708357558.4640212,ghDVqJCSG3QkvQ,Job_7


┌────────┐
│  sum   │
│ int128 │
├────────┤
│    353 │
└────────┘

In [11]:
# QUESTION 3 SOLUTIONS
# create a new pipeline for question 3. use the same schema/dataset_name
pipeline_merge = dlt.pipeline(destination='duckdb', dataset_name='homework')
TABLE_NAME = 'people_merged'
pipeline.run(people_1(), table_name=TABLE_NAME, write_disposition='replace')

# use id as the basis for merging data that refers to the same record
pipeline.run(people_2(), table_name=TABLE_NAME, write_disposition='merge', primary_key='id')

# select the data from the new pipeline
people = conn.sql('SELECT * FROM people_merged').df()
# get the sum of the merged data
sum_merged = conn.sql('SELECT sum(age) AS sum FROM people_merged').df()
# display the results
display(people)
display(sum_merged)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708357832.7277331,7wFRTp93iSQt0w,None
1,2,Person_2,27,City_A,1708357832.7277331,ItOMwsh2Tu6JCA,None
2,5,Person_5,35,City_B,1708357833.4230375,L5kn1kyQ5QoxqA,Job_5
3,8,Person_8,38,City_B,1708357833.4230375,gxd3ZmC7RkMsrQ,Job_8
4,7,Person_7,37,City_B,1708357833.4230375,l0xgoht4SXMxgQ,Job_7
5,4,Person_4,34,City_B,1708357833.4230375,Egnuci+gSkfThA,Job_4
6,3,Person_3,33,City_B,1708357833.4230375,jGtoVEuNhmubRg,Job_3
7,6,Person_6,36,City_B,1708357833.4230375,gpsVE1VRliRZPQ,Job_6


,sum
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX